In [1]:
import os
import pandas as pd
import numpy as np
# import dataframe as df
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import torchmetrics
from torchmetrics.functional import accuracy

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import seaborn as sns
from sklearn.feature_selection import mutual_info_classif as MIC

cuda


In [2]:
X_train = pd.read_csv('../data2.csv')

In [3]:
X_train.head()
X_train.shape
X_train.info()
print(X_train.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14424064 entries, 0 to 14424063
Data columns (total 44 columns):
 #   Column  Dtype  
---  ------  -----  
 0   time    int64  
 1   id      int64  
 2   group   int64  
 3   state   object 
 4   HEOL    float64
 5   HEOR    float64
 6   FP1     float64
 7   FP2     float64
 8   VEOU    float64
 9   VEOL    float64
 10  F7      float64
 11  F3      float64
 12  FZ      float64
 13  F4      float64
 14  F8      float64
 15  FT7     float64
 16  FC3     float64
 17  FCZ     float64
 18  FC4     float64
 19  FT8     float64
 20  T3      float64
 21  C3      float64
 22  CZ      float64
 23  C4      float64
 24  T4      float64
 25  TP7     float64
 26  CP3     float64
 27  CPZ     float64
 28  CP4     float64
 29  TP8     float64
 30  A1      float64
 31  T5      float64
 32  P3      float64
 33  PZ      float64
 34  P4      float64
 35  T6      float64
 36  A2      float64
 37  O1      float64
 38  OZ      float64
 39  O2      float64
 40

In [4]:
import matplotlib.pyplot as plt
# %matplotlib qt
%matplotlib inline

y = X_train['state']
hello = X_train.drop(columns=['time', 'id', 'group', 'state'])
# hello.boxplot()

In [5]:
# Plot EEG Distrobution

FEATURE_COLUMNS = hello.columns.to_list() 

# plt.figure(1)
# plt.subplots_adjust(hspace=0.5)

# for c in FEATURE_COLUMNS:
#     print(c)
#     plt.hist(hello[c], color='skyblue', edgecolor='orange')
#     plt.xlabel(c)
#     plt.ylabel('Frequency')
#     plt.show()

## Mutual Information

In [12]:
from sklearn.metrics import mutual_info_score

def calc_MI(x, y, bins):
    c_xy = np.histogram2d(x, y, bins)[0]
    mi = mutual_info_score(None, None, contingency=c_xy)
    return mi

from scipy.stats import chi2_contingency

def calc_MI(x, y, bins):
    c_xy = np.histogram2d(x, y, bins)[0]
    g, p, dof, expected = chi2_contingency(c_xy, lambda_="log-likelihood")
    mi = 0.5 * g / c_xy.sum()
    return mi

In [13]:
print(calc_MI(hello['F7'], hello['F8'],20))
print(calc_MI(hello['F7'], hello['F8'],10))
print(calc_MI(hello['F7'], hello['F8'],5))

0.001316284664739583
0.001006963928188158
0.00019764138412050664


In [80]:
mi_list = []
print('Calculating...')
for i in FEATURE_COLUMNS:
    for j in FEATURE_COLUMNS:
        if i == j:
            pass
        elif FEATURE_COLUMNS.index(i) >= FEATURE_COLUMNS.index(j):
            pass
        else:
            tmp = calc_MI(hello[i], hello[j],10)
            print([tmp, i, j])
            mi_list.append([tmp, i, j])

Calculating...
[0.0006096925205910991, 'HEOL', 'HEOR']
[0.0006336476292638413, 'HEOL', 'FP1']
[0.0008412546050109456, 'HEOL', 'FP2']
[0.013846928668535171, 'HEOL', 'VEOU']
[0.0008725894309593512, 'HEOL', 'VEOL']
[0.0012356526137141373, 'HEOL', 'F7']
[0.0008647183618287515, 'HEOL', 'F3']
[0.0009347468312166349, 'HEOL', 'FZ']
[0.001097778831100489, 'HEOL', 'F4']
[0.0022738236807686944, 'HEOL', 'F8']
[0.0004153266745161205, 'HEOL', 'FT7']
[0.0005619586756744537, 'HEOL', 'FC3']
[0.0006636272749983334, 'HEOL', 'FCZ']
[0.005020365254875677, 'HEOL', 'FC4']
[0.0006876598087937429, 'HEOL', 'FT8']
[0.00020846078206867885, 'HEOL', 'T3']
[0.0012654929406391386, 'HEOL', 'C3']
[0.0005130005613600404, 'HEOL', 'CZ']
[0.000651806172819851, 'HEOL', 'C4']
[0.0003243429425282344, 'HEOL', 'T4']
[0.0007227729409618063, 'HEOL', 'TP7']
[0.0002797739403481177, 'HEOL', 'CP3']
[0.00021670270402031808, 'HEOL', 'CPZ']
[0.0006874895149725274, 'HEOL', 'CP4']
[0.0008092130151532178, 'HEOL', 'TP8']
[0.0059961347727453

[0.0018960955605786855, 'VEOL', 'FT10']
[0.0009598926009365753, 'VEOL', 'PO1']
[0.0017647925054536382, 'VEOL', 'PO2']
[0.0006827477972362989, 'F7', 'F3']
[0.0014141209772124714, 'F7', 'FZ']
[0.0005532327825707918, 'F7', 'F4']
[0.001006963928188158, 'F7', 'F8']
[0.0004927427607181127, 'F7', 'FT7']
[0.000288701636803406, 'F7', 'FC3']
[0.0007487008370227442, 'F7', 'FCZ']
[0.0069552968328652105, 'F7', 'FC4']
[0.00044830454647552966, 'F7', 'FT8']
[0.003087579592463183, 'F7', 'T3']
[0.0005041242805683725, 'F7', 'C3']
[0.002173259716438858, 'F7', 'CZ']
[0.0017139172928789335, 'F7', 'C4']
[0.0004065403564163592, 'F7', 'T4']
[0.0006827236808463674, 'F7', 'TP7']
[0.0001841577665297545, 'F7', 'CP3']
[0.0001401352820767287, 'F7', 'CPZ']
[0.00021354320653755907, 'F7', 'CP4']
[0.0007185684344897413, 'F7', 'TP8']
[0.0019351766137146538, 'F7', 'A1']
[0.0005155724762060498, 'F7', 'T5']
[0.0006938912728591778, 'F7', 'P3']
[0.0004053091691132108, 'F7', 'PZ']
[0.00020600297888943826, 'F7', 'P4']
[0.000364

[0.004206318356835751, 'FCZ', 'TP8']
[0.0010933413898835812, 'FCZ', 'A1']
[0.0015947407541563926, 'FCZ', 'T5']
[0.001329913592790578, 'FCZ', 'P3']
[0.0018774914405052952, 'FCZ', 'PZ']
[0.0008817063399900756, 'FCZ', 'P4']
[0.0014982440105192662, 'FCZ', 'T6']
[0.0012884094910550403, 'FCZ', 'A2']
[0.0024660461212690993, 'FCZ', 'O1']
[0.0013355889137159447, 'FCZ', 'OZ']
[0.004046595751409449, 'FCZ', 'O2']
[0.0012082354693312254, 'FCZ', 'FT9']
[0.004014489443252834, 'FCZ', 'FT10']
[0.0013657927988937825, 'FCZ', 'PO1']
[0.00426393673720534, 'FCZ', 'PO2']
[0.0005742491530273303, 'FC4', 'FT8']
[0.00016593317585560655, 'FC4', 'T3']
[0.0006179252053532177, 'FC4', 'C3']
[0.0005419079922591135, 'FC4', 'CZ']
[0.0006392482232481559, 'FC4', 'C4']
[0.0002525088821436187, 'FC4', 'T4']
[0.0005538757804562877, 'FC4', 'TP7']
[0.00017605531698427175, 'FC4', 'CP3']
[0.00011139015152663876, 'FC4', 'CPZ']
[0.0004399967318871324, 'FC4', 'CP4']
[0.0008079936715560236, 'FC4', 'TP8']
[0.004342341025060144, 'FC4',

[0.0010126134840187891, 'CP4', 'T5']
[0.0005798508364387754, 'CP4', 'P3']
[0.001074566138795905, 'CP4', 'PZ']
[0.0010878683124816893, 'CP4', 'P4']
[0.0010385898194890599, 'CP4', 'T6']
[0.0005675389742562526, 'CP4', 'A2']
[0.000793003579901129, 'CP4', 'O1']
[0.001060287628316396, 'CP4', 'OZ']
[0.0020705852303391954, 'CP4', 'O2']
[0.0006032584988516301, 'CP4', 'FT9']
[0.002140669247565077, 'CP4', 'FT10']
[0.0012570052603319709, 'CP4', 'PO1']
[0.002190035309930703, 'CP4', 'PO2']
[0.001138362921454486, 'TP8', 'A1']
[0.001797858009378019, 'TP8', 'T5']
[0.001220213121776861, 'TP8', 'P3']
[0.0021332119881068284, 'TP8', 'PZ']
[0.000869239256161233, 'TP8', 'P4']
[0.0019553196356689733, 'TP8', 'T6']
[0.0013350637080380117, 'TP8', 'A2']
[0.002395078858467382, 'TP8', 'O1']
[0.001255365096301891, 'TP8', 'OZ']
[0.004119429436844924, 'TP8', 'O2']
[0.0012804583370304905, 'TP8', 'FT9']
[0.004414475642897022, 'TP8', 'FT10']
[0.0012519988280599366, 'TP8', 'PO1']
[0.006929016432907249, 'TP8', 'PO2']
[0.00

In [95]:
mi_df = pd.DataFrame(mi_list, columns=['value', 'C1', 'C2'])        
mi_df = mi_df.reindex(mi_df.value.abs().sort_values().index)
print(mi_df.info)

<bound method DataFrame.info of         value    C1    C2
357  0.000086    F8   CPZ
463  0.000111   FC4   CPZ
510  0.000114    T3   CPZ
552  0.000115    CZ   CP3
412  0.000115   FC3   CPZ
..        ...   ...   ...
212  0.016541  VEOL    O1
707  0.017252    T5    O1
64   0.025839  HEOR    T5
70   0.055735  HEOR    O1
42   0.079232  HEOR  VEOL

[780 rows x 3 columns]>


In [117]:
copy = mi_df.copy()
print(mi_df)

        value    C1    C2
357  0.000086    F8   CPZ
463  0.000111   FC4   CPZ
510  0.000114    T3   CPZ
552  0.000115    CZ   CP3
412  0.000115   FC3   CPZ
..        ...   ...   ...
212  0.016541  VEOL    O1
707  0.017252    T5    O1
64   0.025839  HEOR    T5
70   0.055735  HEOR    O1
42   0.079232  HEOR  VEOL

[780 rows x 3 columns]


In [118]:
removal_list = []
num = 0
while not copy.empty:
#     print('LAST:', copy.iloc[-1]['C1'], copy.iloc[-1]['C2'])
    a = copy.iloc[-1]['C1']
    b = copy.iloc[-1]['C2']
    
    if (a in removal_list or b in removal_list):
        pass
    else:
        removal_list.append(a)
        if(num == 38):
            removal_list.append(b)
            num += 1
        print('num:', num, removal_list)
        num += 1
        
    copy.drop(copy.tail(1).index,inplace=True)

num: 0 ['HEOR']
num: 1 ['HEOR', 'T5']
num: 2 ['HEOR', 'T5', 'VEOL']
num: 3 ['HEOR', 'T5', 'VEOL', 'VEOU']
num: 4 ['HEOR', 'T5', 'VEOL', 'VEOU', 'F8']
num: 5 ['HEOR', 'T5', 'VEOL', 'VEOU', 'F8', 'FZ']
num: 6 ['HEOR', 'T5', 'VEOL', 'VEOU', 'F8', 'FZ', 'F7']
num: 7 ['HEOR', 'T5', 'VEOL', 'VEOU', 'F8', 'FZ', 'F7', 'TP8']
num: 8 ['HEOR', 'T5', 'VEOL', 'VEOU', 'F8', 'FZ', 'F7', 'TP8', 'FC3']
num: 9 ['HEOR', 'T5', 'VEOL', 'VEOU', 'F8', 'FZ', 'F7', 'TP8', 'FC3', 'T4']
num: 10 ['HEOR', 'T5', 'VEOL', 'VEOU', 'F8', 'FZ', 'F7', 'TP8', 'FC3', 'T4', 'HEOL']
num: 11 ['HEOR', 'T5', 'VEOL', 'VEOU', 'F8', 'FZ', 'F7', 'TP8', 'FC3', 'T4', 'HEOL', 'FT10']
num: 12 ['HEOR', 'T5', 'VEOL', 'VEOU', 'F8', 'FZ', 'F7', 'TP8', 'FC3', 'T4', 'HEOL', 'FT10', 'O2']
num: 13 ['HEOR', 'T5', 'VEOL', 'VEOU', 'F8', 'FZ', 'F7', 'TP8', 'FC3', 'T4', 'HEOL', 'FT10', 'O2', 'FC4']
num: 14 ['HEOR', 'T5', 'VEOL', 'VEOU', 'F8', 'FZ', 'F7', 'TP8', 'FC3', 'T4', 'HEOL', 'FT10', 'O2', 'FC4', 'FCZ']
num: 15 ['HEOR', 'T5', 'VEOL', 'VEOU', 

In [ ]:
# 移除順序
# ['HEOR', 'T5', 'VEOL', 'VEOU', 'F8', 'FZ', 'F7', 'TP8', 'FC3', 'T4', 'HEOL', 'FT10', 'O2', 'FC4', 'FCZ', 'CP3', 'FP2', 'TP7', 'C3', 'F3', 'PZ', 'F4', 'O1', 'FT7', 'CP4', 'FP1', 'C4', 'T6', 'FT8', 'CZ', 'PO1', 'OZ', 'A2', 'A1', 'T3', 'P4', 'P3', 'FT9', 'CPZ', 'PO2']